In [4]:
import pandas as pd
from pandas.io.formats.style import Styler
from pybatfish.client.session import Session
from pybatfish.datamodel import *
from pybatfish.datamodel.answer import *
from pybatfish.datamodel.flow import *
from pybatfish.util import get_html
import json
import os

In [5]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
# Prevent rendering text between '$' as MathJax expressions
pd.set_option("display.html.use_mathjax", False)

def display_tr(tr):
    display(Styler(tr).format(get_html).set_properties(**{"text-align": "left", "vertical-align": "top"}))

In [6]:
bf = Session('localhost')
# Electnet Policy Test
bf.set_network('elecnet')
bf.init_snapshot("networks/elecnet", '1', overwrite=True)

'1'

In [7]:
bf.q.nodeProperties().answer().frame()

,Node,AS_Path_Access_Lists,Authentication_Key_Chains,Community_Match_Exprs,Community_Set_Exprs,Community_Set_Match_Exprs,Community_Sets,Configuration_Format,DNS_Servers,DNS_Source_Interface,Default_Cross_Zone_Action,Default_Inbound_Action,Domain_Name,Hostname,IKE_Phase1_Keys,IKE_Phase1_Policies,IKE_Phase1_Proposals,IP6_Access_Lists,IP_Access_Lists,IPsec_Peer_Configs,IPsec_Phase2_Policies,IPsec_Phase2_Proposals,Interfaces,Logging_Servers,Logging_Source_Interface,NTP_Servers,NTP_Source_Interface,PBR_Policies,Route6_Filter_Lists,Route_Filter_Lists,Routing_Policies,SNMP_Source_Interface,SNMP_Trap_Servers,TACACS_Servers,TACACS_Source_Interface,VRFs,Zones
0,eme-cht,[],[],[],[],[],[],HOST,[],None,PERMIT,PERMIT,None,eme-cht,[],[],[],[],"['filter::FORWARD', 'filter::INPUT', 'filter::OUTPUT']",[],[],[],['eth0'],[],None,[],None,[],[],[],[],None,[],[],None,['default'],[]
1,eme-edn,[],[],[],[],[],[],HOST,[],None,PERMIT,PERMIT,None,eme-edn,[],[],[],[],"['filter::FORWARD', 'filter::INPUT', 'filter::OUTPUT']",[],[],[],['eth0'],[],None,[],None,[],[],[],[],None,[],[],None,['default'],[]
2,pamnode0,[],[],[],[],[],[],HOST,[],None,PERMIT,PERMIT,None,pamnode0,[],[],[],[],"['filter::FORWARD', 'filter::INPUT', 'filter::OUTPUT']",[],[],[],['eth0'],[],None,[],None,[],[],[],[],None,[],[],None,['default'],[]
3,percore-2,[],[],[],[],[],[],CUMULUS_CONCATENATED,[],None,PERMIT,PERMIT,None,percore-2,[],[],[],[],"['filter::FORWARD', 'filter::INPUT', 'filter::OUTPUT']",[],[],[],"['eth0', 'eth1', 'eth2', 'eth3', 'eth4', 'eth5', 'eth6', 'lo']",[],None,[],None,[],[],[],['~OSPF_EXPORT_POLICY:default~'],None,[],[],None,['default'],[]
4,echnode1,[],[],[],[],[],[],HOST,[],None,PERMIT,PERMIT,None,echnode1,[],[],[],[],"['filter::FORWARD', 'filter::INPUT', 'filter::OUTPUT']",[],[],[],['eth0'],[],None,[],None,[],[],[],[],None,[],[],None,['default'],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,ess-kng,[],[],[],[],[],[],HOST,[],None,PERMIT,PERMIT,None,ess-kng,[],[],[],[],"['filter::FORWARD', 'filter::INPUT', 'filter::OUTPUT']",[],[],[],['eth0'],[],None,[],None,[],[],[],[],None,[],[],None,['default'],[]
139,eme-nkr,[],[],[],[],[],[],HOST,[],None,PERMIT,PERMIT,None,eme-nkr,[],[],[],[],"['filter::FORWARD', 'filter::INPUT', 'filter::OUTPUT']",[],[],[],['eth0'],[],None,[],None,[],[],[],[],None,[],[],None,['default'],[]
140,inkanode2,[],[],[],[],[],[],HOST,[],None,PERMIT,PERMIT,None,inkanode2,[],[],[],[],"['filter::FORWARD', 'filter::INPUT', 'filter::OUTPUT']",[],[],[],['eth0'],[],None,[],None,[],[],[],[],None,[],[],None,['default'],[]
141,eme-rch,[],[],[],[],[],[],HOST,[],None,PERMIT,PERMIT,None,eme-rch,[],[],[],[],"['filter::FORWARD', 'filter::INPUT', 'filter::OUTPUT']",[],[],[],['eth0'],[],None,[],None,[],[],[],[],None,[],[],None,['default'],[]


In [8]:
# Policy Tests

# From BarrowIsland node (Wind, Company A) to frknode1 (Hydro, A), is allowed
# root@brwnode0:/tmp/pycore.1/brwnode0.conf# ping 10.25.56.149
tr = bf.q.traceroute(startLocation="brwnode0", headers=HeaderConstraints(dstIps="10.25.56.149")).answer().frame()
display_tr(tr)


,Flow,Traces,TraceCount
0,Start Location: brwnode0Src IP: 10.25.11.81Src Port: 49152Dst IP: 10.25.56.149Dst Port: 33434IP Protocol: UDP,"DENIED_OUT1. node: brwnode0 ORIGINATED(default) FORWARDED(Forwarded out interface: eth0 with resolved next-hop IP: 10.25.11.96, Routes: [static (Network: 0.0.0.0/0, Next Hop: interface eth0 ip 10.25.11.96)]) PERMITTED(filter::OUTPUT (EGRESS_FILTER)) TRANSMITTED(eth0)2. node: brwnodegw RECEIVED(eth2) FORWARDED(Forwarded out interface: eth0 with resolved next-hop IP: 10.25.205.84, Routes: [ospf (Network: 10.25.56.0/24, Next Hop: interface eth0 ip 10.25.205.84)]) DENIED(filter::FORWARD (EGRESS_FILTER))",1


In [11]:
# From BarrowIsland node (Wind, Company A) to cbpd (Wind, C), block
# root@brwnode0:/tmp/pycore.1/brwnode0.conf# ping 10.25.69.221
tr = bf.q.traceroute(startLocation="brwnode0", headers=HeaderConstraints(dstIps="10.25.205.84")).answer().frame()
display_tr(tr)


,Flow,Traces,TraceCount
0,Start Location: brwnode0Src IP: 10.25.11.81Src Port: 49152Dst IP: 10.25.205.84Dst Port: 33434IP Protocol: UDP,"DENIED_OUT1. node: brwnode0 ORIGINATED(default) FORWARDED(Forwarded out interface: eth0 with resolved next-hop IP: 10.25.11.96, Routes: [static (Network: 0.0.0.0/0, Next Hop: interface eth0 ip 10.25.11.96)]) PERMITTED(filter::OUTPUT (EGRESS_FILTER)) TRANSMITTED(eth0)2. node: brwnodegw RECEIVED(eth2) FORWARDED(Forwarded out interface: eth0, Routes: [connected (Network: 10.25.205.0/24, Next Hop: interface eth0)]) DENIED(filter::FORWARD (EGRESS_FILTER))",1


In [12]:
# And why was it blocked? (manually putting in IP, node and filter info)
t = bf.q.testFilters(headers=HeaderConstraints(srcIps='10.25.11.81', dstIps='10.25.56.149'), nodes='cbpdnodegw', filters='filter::FORWARD').answer().frame()
display_tr(t)

,Node,Filter_Name,Flow,Action,Line_Content,Trace
0,cbpdnodegw,filter::FORWARD,Start Location: cbpdnodegwSrc IP: 10.25.11.81Src Port: 49152Dst IP: 10.25.56.149Dst Port: 80IP Protocol: TCP (SYN),DENY,default,Matched line default


In [13]:
# Show interface properties
bf.q.interfaceProperties().answer().frame()
# There are interfaces without ip address, bug?

,Interface,Access_VLAN,Active,Admin_Up,All_Prefixes,Allowed_VLANs,Auto_State_VLAN,Bandwidth,Blacklisted,Channel_Group,Channel_Group_Members,DHCP_Relay_Addresses,Declared_Names,Description,Encapsulation_VLAN,HSRP_Groups,HSRP_Version,Inactive_Reason,Incoming_Filter_Name,MLAG_ID,MTU,Native_VLAN,Outgoing_Filter_Name,PBR_Policy_Name,Primary_Address,Primary_Network,Proxy_ARP,Rip_Enabled,Rip_Passive,Spanning_Tree_Portfast,Speed,Switchport,Switchport_Mode,Switchport_Trunk_Encapsulation,VRF,VRRP_Groups,Zone_Name
0,adlcore-1[eth0],None,True,True,['10.21.9.193/30'],,True,10000000000.0,False,None,[],[],[],None,None,[],None,,None,None,1500,None,filter::FORWARD,None,10.21.9.193/30,10.21.9.192/30,False,False,False,False,None,False,NONE,DOT1Q,default,[],None
1,adlcore-1[eth1],None,True,True,['10.22.27.115/29'],,True,10000000000.0,False,None,[],[],[],None,None,[],None,,None,None,1500,None,filter::FORWARD,None,10.22.27.115/29,10.22.27.112/29,False,False,False,False,None,False,NONE,DOT1Q,default,[],None
2,adlcore-1[eth2],None,True,True,['10.22.11.73/29'],,True,10000000000.0,False,None,[],[],[],None,None,[],None,,None,None,1500,None,filter::FORWARD,None,10.22.11.73/29,10.22.11.72/29,False,False,False,False,None,False,NONE,DOT1Q,default,[],None
3,adlcore-1[eth3],None,True,True,['10.10.22.25/30'],,True,10000000000.0,False,None,[],[],[],None,None,[],None,,None,None,1500,None,filter::FORWARD,None,10.10.22.25/30,10.10.22.24/30,False,False,False,False,None,False,NONE,DOT1Q,default,[],None
4,adlcore-1[eth4],None,True,True,['10.10.10.198/30'],,True,10000000000.0,False,None,[],[],[],None,None,[],None,,None,None,1500,None,filter::FORWARD,None,10.10.10.198/30,10.10.10.196/30,False,False,False,False,None,False,NONE,DOT1Q,default,[],None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,wwanodegw[lo],None,True,True,['127.0.0.1/8'],,True,8000000000.0,None,None,[],[],[],None,None,[],None,,None,None,65536,None,filter::FORWARD,None,127.0.0.1/8,127.0.0.0/8,False,False,False,False,None,False,NONE,DOT1Q,default,[],None
402,yrv-gw[eth0],None,True,True,['10.22.217.196/29'],,True,10000000000.0,False,None,[],[],[],None,None,[],None,,None,None,1500,None,filter::FORWARD,None,10.22.217.196/29,10.22.217.192/29,False,False,False,False,None,False,NONE,DOT1Q,default,[],None
403,yrv-gw[eth1],None,True,True,['10.22.221.181/29'],,True,10000000000.0,False,None,[],[],[],None,None,[],None,,None,None,1500,None,filter::FORWARD,None,10.22.221.181/29,10.22.221.176/29,False,False,False,False,None,False,NONE,DOT1Q,default,[],None
404,yrv-gw[eth2],None,True,True,['10.23.208.165/29'],,True,10000000000.0,False,None,[],[],[],None,None,[],None,,None,None,1500,None,filter::FORWARD,None,10.23.208.165/29,10.23.208.160/29,False,False,False,False,None,False,NONE,DOT1Q,default,[],None


In [15]:
tr = bf.q.traceroute(startLocation="wlwnode0", headers=HeaderConstraints(dstIps="10.25.212.47")).answer().frame()
tr

,Flow,Traces,TraceCount
0,start=wlwnode0 [10.25.21.123:49152->10.25.212.47:33434 UDP],"[((ORIGINATED(default), FORWARDED(Forwarded out interface: eth0 with resolved next-hop IP: 10.25.21.72, Routes: [static (Network: 0.0.0.0/0, Next Hop: interface eth0 ip 10.25.21.72)]), PERMITTED(filter::OUTPUT (EGRESS_FILTER)), TRANSMITTED(eth0)), (RECEIVED(eth1), FORWARDED(Forwarded out interface: eth0, Routes: [connected (Network: 10.25.212.0/25, Next Hop: interface eth0)]), DENIED(filter::FORWARD (EGRESS_FILTER))))]",1


In [16]:
bf.q.layer3Edges().answer().frame()

,Interface,IPs,Remote_Interface,Remote_IPs
0,eme-brd[eth0],['10.23.180.211'],ess-brd[eth0],['10.23.180.209']
1,eme-byo[eth0],['10.23.119.122'],ess-byo[eth0],['10.23.119.125']
2,eme-cht[eth0],['10.23.142.134'],ess-cht[eth0],['10.23.142.133']
3,eme-edn[eth0],['10.23.4.148'],ess-edn[eth0],['10.23.4.149']
4,eme-fre[eth0],['10.23.33.126'],ess-fre[eth0],['10.23.33.125']
5,eme-joo[eth0],['10.23.39.110'],ess-joo[eth0],['10.23.39.106']
6,eme-kng[eth0],['10.23.191.3'],ess-kng[eth0],['10.23.191.5']
7,eme-mbr[eth0],['10.23.147.26'],ess-mbr[eth0],['10.23.147.28']
8,eme-nkr[eth0],['10.23.91.126'],ess-nkr[eth0],['10.23.91.124']
9,eme-plm[eth0],['10.23.82.42'],ess-plm[eth0],['10.23.82.41']
